 # TP6: Apprentissage profond(Deep learning)
 ## TP6.0 Le perceptron from scratch: Régression linéaire/logistique  
 Dans ce TP, nous explorerons le fonctionnement interne d'un neurone.


#### Travail demandé :
Objectif : démystifier la partie mathématiques dus réseaux de neurones
- Définir un perceptron simple
- Ecrire les fonctions qui permettet de faire tous les calculs 

On utilisera  deux datasets : le titanic et le housing. Un portant sur une classification l'aure sur une régression.
Les programmes sont identifques seuls le calcul de l'activation, la loss et les dw et db 
qui changent légérement. 
<span style="color:green">  L'architecture du réseau à un réel impact sur les résultats </span>

In [141]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, r2_score

## Neurone/ Perceptron  

### 1. Sélection des données 

### Dataset pour la classification

In [142]:
# le dataset titanic 
import pandas as pd 
titanic = pd.read_csv('../../data/titanic.csv')

# On prend juste une partie du fichier.
titanic = titanic[['Survived', 'Pclass', 'Sex', 'Age']]
#  on regrade les chance de Survived  en fonction des autres features.
X_features=['Pclass', 'Sex', 'Age']
titanic.dropna(axis=0, inplace=True)
titanic['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
titanic.head()

y = titanic['Survived'] # récupérer la colonne survived et la mettre dans y

# récuperer le reste des données dans X utiliser la fonction titanic.drop ???, ??? )
X = titanic.drop('Survived', axis=1)
print(X.shape,y.shape)

(714, 3) (714,)


## 2. Préparation des données

#### Split des données 

In [143]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# reshape Y 
y_train = y_train.values.reshape(-1,1) # -1 toutes les lignes, 1 une colonne

In [144]:
print(X_train.shape,y_train.shape)

(499, 3) (499, 1)


###   Normalisation des données
Rappel : Il est primordial de normaliser les données. Vous pourrez vérifier les performances de vos modèles avec et sans données normalisées

In [145]:
## On peut utiliser une simple normalisation (X - moyenne)/ecart type)
# C'est une standardisation la même que StandardScaler de scikit learn
 
mean = X_train.mean()
std  = X_train.std()
X_train = (X_train - mean) / std

# Normalisation des données de test
X_test  = (X_test  - mean) / std

#display(X_train.describe().style.format("{0:.2f}").set_caption("After normalization :"))
display(X_train.head(3).style.format("{0:.2f}").set_caption("Few lines of the dataset :"))

,Pclass,Sex,Age
37,0.93,-0.75,-0.63
58,-0.25,1.34,-1.70
57,0.93,-0.75,-0.12


## 3. Construction (définition) du modèle 
On construit un modèle de régression linéaire. 
<figure>
    <img src="../images/perceptron.png"  style="width:440px;height:200px;" >
</figure>

$$ a=\sigma(z) = \frac{1}{1+\exp(-z)}$$




### 3.1. Fonctions du modele
#### Petit rappel : on manipule directement les matrices et les vecteurs. Pas besoin de faire des boucles sur l'ensemble des valeurs, .... 
#### Cette question a déjà été traitée dans un des TP précédents (vectorisation, voir lab_utils_multi.py TP2)

### Questions : 
Ecrire les fonctions ci-dessous

### Initialisation des paramètres W et b

In [146]:
# Combien de paramètres 
def initialisation(X): # initialise les paramètres W et b
    W = np.random.randn(1,X.shape[1])   # X.shape[1] donne le nombre de paramètres, W = np.random.randn(1, X.shape[1]) car on a un seul neurone et X.shape[1] paramètres
    b = np.random.randn(1) # b = np.random.randn(1) car on a un seul neurone
    return (W, b)

### Calcul activation du modèle (Z et A)

In [147]:
# Calcul activation du modèle Z et A 
def model(X, W, b): # X représente les données, W et b les paramètres du modèle
    Z =  np.dot(X,W)+b# Z est égal a X*W+b
    A = 1/(1+np.exp(-Z))# A est égal à la sigmoide de Z, la fonction sigmoide = 1 divisé par 1 + exp(-Z), Sigmoide = 1/(1+np.exp(-Z))
    return A

### Calcul de la Loss
$$loss=1/m * \sum (y * log(A) - (1 - y) * log(1 - A))$$

In [148]:
# Calcler la Loss (la MSE
def log_loss(A, y):  # A est l'activation du modèle, y les vraies valeurs
    # Il faut calculer la loss, voici la fonction : loss=1/m * \sum (y * log(A) - (1 - y) * log(1 - A))
    # m = y.shape[0] # nombre d'exemples
    # sum (y * log(a) représente la somme des y * log(a) pour chaque exemple
    return 1/y.shape[0] * y*np.log(A) - (1-y) * np.log(1-A)
    

### Calcul des gradients

In [149]:
def gradients(A, X, y):
    # pour calculer le gradient il faut : dw qui est égal a 1/m * X * (A - y) et db qui est égal a 1/m * (A - y)
    # m = y.shape[0] # nombre d'exemples 
    dW = 1/y[0] * X * (A-y)
    db = 1/y[0] * (A-y)
    return dW, db

### Modifier w et b

In [150]:
def update(dW, db, W, b, learning_rate): # learning_rate est le taux d'apprentissage
    W = W - learning_rate*dW # W représente les paramètres du modèle et est égal à W - learning_rate * dW
    b = b-learning_rate*db # b représente les paramètres du modèle et est égal à b - learning_rate * db
    return (W, b)

### Prediction 

In [151]:
def predict(X, W, b): # prédict permet de prédire quelque chose en fonction de nouvelles données qui ne sont pas dans le dataset
    A = model(X, W, b)
    return A >= 0.5 # classification, retourne vrai si A est supérieur ou égal à 0.5, faux sinon
    

### Définition du neurone et de ses fonctions : mettre tout dans une même fonction
- initialiser les paramètres du modèle
- définir les hyperparamètres : learning_rate, nombre_d'itérations
- commencer les traitements (dans une boucle) 
    - appel da la fonction qui calcule l'activation
    - caculer l'erreur (pour la visualiser) 
    - calculer les gradients
    - effectuer les modification des paramètres w et b
    - réitérer
- une fois le modèle appris
    - commencer la prédiction
    - visualiser la Loss et les performances


In [152]:
def perceptron(X, y, learning_rate, n_iter):
    # initialisation W, b
    W, b = initialisation(X)
    Loss = []
    Les_W =[]
    # itérer l'apprentissage 
    for i in range(n_iter):
        #Calculer l'activation du neurone
        A = model(X,W,b) # appel de la fonction model A = model(X, W, b)
        #Calculer la Loss mettre dans une liste
        Loss.append(log_loss(A, y))
        # Calul des gradients
        dW, db = gradients(A, X, y) # appel de la fonction gradients dW, db = gradients(A, X, y)
        
        # Modifier les paramètres
        W, b = update(dW, db, W, b, learning_rate) # appel de la fonction update W, b = update(dW, db, W, b, learning_rate
        
    return (W, b, Loss)
    

In [153]:
learning_rate = 0.5
n_iter = 10000
W, b , Loss = perceptron(X_train, y_train, learning_rate,n_iter )

iteration 0
iteration 100
iteration 200
iteration 300
iteration 400
iteration 500
iteration 600
iteration 700
iteration 800
iteration 900
iteration 1000
iteration 1100
iteration 1200
iteration 1300
iteration 1400
iteration 1500
iteration 1600
iteration 1700
iteration 1800
iteration 1900
iteration 2000
iteration 2100
iteration 2200
iteration 2300
iteration 2400
iteration 2500
iteration 2600
iteration 2700
iteration 2800
iteration 2900
iteration 3000
iteration 3100
iteration 3200
iteration 3300
iteration 3400
iteration 3500
iteration 3600
iteration 3700
iteration 3800
iteration 3900
iteration 4000
iteration 4100
iteration 4200
iteration 4300
iteration 4400
iteration 4500
iteration 4600
iteration 4700
iteration 4800
iteration 4900
iteration 5000
iteration 5100
iteration 5200
iteration 5300
iteration 5400
iteration 5500
iteration 5600
iteration 5700
iteration 5800
iteration 5900
iteration 6000
iteration 6100
iteration 6200
iteration 6300
iteration 6400
iteration 6500
iteration 6600
iterati

#### Calcul des prédictions et les performances appropriées (Accuracy ou r2_score)

In [154]:
## Accuracy pour la classification
y_pred_train= predict(X_train,W,b)# y_pred_train est égal à predict(X_train, W, b) # on utilise x_train pour prédire y_train
y_pred = (X_test,W,b)# y_pred est égal à predict(X_test, W, b)  # on utilise x_test pour prédire y_test

# Calcul de l'accuracy 
print("Accuracy train", accuracy_score(y_train, y_pred_train), "Accuracy Test", accuracy_score(y_test, y_pred))


(499, 1) (499, 1)
(215,) (215, 1)
Accuracy train 0.7995991983967936 Accuracy Test 0.7674418604651163


### Visualiser la Loss

In [158]:
plt.plot(Loss)
plt.show()

KeyboardInterrupt: 

## Questions
L'objectif dec cette partie est d'évaluer l'impact du taux d'apprentissage (learning_rate). 
- 1. Reprendre l'algorithme d'apprentissage, mettre en place un learning_rate dynamique, qui est mis à jour chaque fois que l'iteration, $t$ (qui correspond à l'indice $i$ dans le programme principal) prend une valeur multiple de $10$, $100$, vous pourrez utiliser la forme suivante : $\alpha=\frac{\alpha}{\sqrt(t+1)}$, t étant l'itération. 
- 2. Comparer les courbes des Loss obtenues pour chacun de ces cas (learning_rate constant et learning_rate dynamique avec $t$ aleur multiple de $10$, $100$).
- 3. On souhaite arrêter l'apprentissage soit quand l'erreur est inférieure à 0.1 ou quand on atteint le nombre diitérations max, rajouter cette contrainte à l'algortime   
- 4. Refaire l'exercice en prenant les données du fichier smoking (du contôle). Le dataset est dans la cellule ci dessous. 


In [ ]:
# Lecture du dataset n permet de limiter le nombre de lignes à lire
# Pour faciliter les tests
import pandas as pd 
def load_data(n):
    data = pd.read_csv('../../data/train.csv')
    return data[0:n]

# le -1 du load_data(-1) veut dire on prend toutes les lignes 
data=load_data(-1)
data.dropna(axis=0, inplace=True)
y = data['smoking'] # récupérer la colonne survived et la mettre dans y
# récuperer le reste des données dans X utiliser la fonction titanic.drop ???, ??? )
X = data.drop('smoking', axis=1)